In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import json
import sys
from pathlib import Path
import re 

import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid

root_path = Path('/dev/projects/cds/')

if (path := str(root_path.resolve())) not in sys.path:
    sys.path.append(path)

from api.blpw import BlpQuery
from api.data.base import data_path
from api.data.curves import curves
from api.data.utils import add_pcs, parse_tenor
from api.pricing.cds import get_isda_cds
from api.data.processing import apply_function
from api.pricing.utils import (
    get_cds_specs,
    get_cds_curve,
    get_cds_spreads,
    get_risk_free_curve,
    get_cds_implied_price,
    get_curve,
)

In [6]:
# Technically, the model requires the discount factor to be 1 at spot;
# but we can't do that and also have the discount curve extend back to
# the trade date.  For the time being, we'll keep discount = 1 at trade.
# The results match anyway.

In [7]:
securities = [
    'XS1968706876 Corp',
    'XS2109813142 Corp',
    'US195325CX13 Corp',
    'US195325DX04 Corp',
    'XS1558078736 Corp',
    'XS2391398174 Corp',
    'XS1843443356 Corp',
    'US900123CM05 Corp',
    'CMEX1U5 Corp',
    'XS2289588167 Corp',
    'CCHIL1U5 Corp',
    'US168863DU93 Corp',
    'US718286CB15 Corp',
    'US718286CJ41 Corp',
    'US715638BM30 Corp',
    'US715638DQ26 Corp',
    'XS2050933626 Corp',
    'XS1263139856 Corp',
    'XS1245432742 Corp',
    'XS2330514899 Corp',
    'XS2178857285 Corp',
    'XS1694217495 Corp',
    'XS1936302949 Corp',
    'XS1807174393 Corp',
    'XS1959337749 Corp',
    'US195325DP79 Corp',
    'USM88269US88 Corp',
    'XS2445169985 Corp',
    'US836205BA15 Corp',
    'US836205AV60 Corp',
    'US900123AL40 Corp',
    'XS2455985569 Corp',
    '91282CEC1 Corp',
    'XS2455984679 Corp',
    '91282CDY4 Corp',
    'US715638DP43 Corp',
    'US718286CQ83 Corp',
    'US718286CR66 Corp',
    'USY6972HLP91 Corp',
    'US760942BA98 Corp',
    'US760942BB71 Corp',
    'US455780DJ24 Corp',
    'XS1120709826 Corp',
    'CTURK1U3 Corp',
    'US105756CB40 Corp',
    'CBRZ1U5 Corp',
    'US445545AF36 Corp',
    'XS2388586583 Corp',
    'XS1775617464 Corp',
    'US455780CS32 Corp',
    'US455780CT15 Corp',
    'XS2364200514 Corp',
    'XS1313004928 Corp',
    'US715638DF60 Corp',
    'CEGY1U5 Corp',
    'US91087BAL45 Corp',
    'CCOL1U5 Corp',
    'US040114HT09 Corp',
    'US040114HV54 Corp',
    'XS2468421248 Corp',
    'US900123CL22 Corp',
    'US91087BAN01 Corp',
    'US91087BAS97 Corp',
    'XS2159975700 Corp',
    'XS2159975882 Corp',
    'XS2225210330 Corp',
    'XS1696899035 Corp',
    'XS2446175577 Corp',
    'USY7141BAA18 Corp',
    'USY7141BAB90 Corp',
    'XS2259191430 Corp',
    'CPERU1U5 Corp',
    'CSOAF1U5 Corp',
    'US836205BC70 Corp',
    'US836205BE37 Corp',
    'BV8695652 Corp',
    'US195325EA91 Corp',
    'CTURK1U5 Corp',
    'US836205BB97 Corp',
    'XS1892141620 Corp',
    'US698299BL70 Corp',
    'US698299AW45 Corp',
    'XS2109766472 Corp',
    'US455780CW44 Corp',
    'US455780DG84 Corp',
    'XS2270577344 Corp',
    'XS2270576619 Corp',
    'XS2434895806 Corp',
    'XS1028952403 Corp',
    'XS1781710626 Corp',
    'XS1843435840 Corp',
    'USP3579ECB13 Corp',
    'USP3579ECG00 Corp',
    'USP3699PGH49 Corp',
    'XS1108847531 Corp',
    'XS2325747637 Corp',
    'XS1236685613 Corp',
    'XS2388562139 Corp',
    'USP75744AK10 Corp',
    'USP75744AB11 Corp',
    'XS2472852610 Corp',
    'XS1382696398 Corp',
    'XS2079846635 Corp',
    'US718286BG11 Corp',
    'US718286CL96 Corp',
    'XS1968714623 Corp',
    'US836205AY00 Corp',
    'CTUR1U10 Corp',
    'CY002690 Corp',
    'XS2434896010 Corp',
    'XS2447602793 Corp',
    'XS2485248806 Corp',
    'XS2291692890 Corp',
    'XS2388586401 Corp',
    'US71567RAV87 Corp',
    'XS2487342649 Corp',
    'US900123CB40 Corp',
    'XS1057340009 Corp',
    'US900123CA66 Corp',
    'US41809JAA34 Corp',
    'XS2010026305 Corp',
    'XS2010026487 Corp',
    'XS2485249523 Corp',
    'US105756BX78 Corp',
    'CTURK1U1 Corp',
    'US195325BQ70 Corp',
    'XS2280637039 Corp',
    'US900123BJ84 Corp',
    'XS1717013095 Corp',
    'XS1312891549 Corp',
    'US900123CF53 Corp',
    'XS1821416234 Corp',
    'XS2258400162 Corp',
    'USP7S08VCA70 Corp',
    'US71654QBX97 Corp',
    'XS2210006339 Corp',
    'XS2386638733 Corp',
    'XS2391394348 Corp',
    'XS1837994794 Corp',
    'XS1369323149 Corp',
    'US31424EAD40 Corp',
    'US836205AX27 Corp',
    'XS1843435766 Corp',
    'XS2176899701 Corp',
    'XS0505478684 Corp',
    'XS1106137687 Corp',
    'XS2391395154 Corp',
    'XS2451768720 Corp',
    'XS2297221405 Corp',
]

#### Functions

In [166]:
def get_overrides(scen, row):
    key = scen['override_value']
    return [s['fn'](row[key]) for s in scen['scenarios']]

In [165]:
def get_stressed(mkt, fields, scenarios):
    row = mkt
    res = bq.bdph(
        securities=[row['security']],
        fields=fields,
        id_override=get_overrides(scenarios, row),
        id_override_field=scenarios['override_field']

    )
    return res.set_axis([scen['name'] for scen in scenarios['scenarios']]).rename_axis('scenario')
    

#### Settings

In [9]:
dt_fmt = '%Y-%m-%d'
bbg_dt_fmt = '%Y%m%d'

#### Bloomberg Connection

In [10]:
bq = BlpQuery(timeout=50000).start()

#### Common

In [124]:
ref_fields = ['security_des', 'security_typ']

In [125]:
ref = bq.bdp(
    securities=securities,
    fields=ref_fields
)

#### Bonds

In [ ]:
bond_mkt_fields = [
    'yas_bond_px',
    'yas_zspread',
    'yas_mod_dur'
]

In [126]:
bond_ref = ref.loc[~(ref['security_typ'] == 'CREDIT DEFAULT SWAP'), :]

In [128]:
bond_pcs = 'BGN'
bond_sep = '@'

bond_mkt = bq.bdp(
    securities=bond_ref['security'].to_list() if not bond_pcs else add_pcs(bond_ref['security'], source=bond_pcs, sep=bond_sep),
    fields=bond_mkt_fields
)
if bond_pcs:
    bond_mkt['security'] = bond_mkt['security'].str.replace(f'{bond_sep}{bond_pcs}', '')

In [72]:
z_spread_scenarios = {
    'override_field': 'yas_zspread',
    'override_value': 'yas_zspread',
    'scenarios': [
        {
            'name': 'double z',
            'fn': lambda x: x * 2
        },
        {
            'name': 'half z',
            'fn': lambda x: x / 2
        },
    ]
} 

In [129]:
n = 5
bond_res = []
for index, row in bond_mkt.iterrows():
    if index + 1 <= n:
        bond_res.append(get_stressed_bond(row, ['yas_bond_px', 'yas_mod_dur'], z_spread_scenarios))

In [131]:
bond_base = pd.concat([bond_mkt.set_index('security')], keys=['base'], axis=1).rename_axis(['scenario', None], axis=1)
bond_stress = pd.concat(bond_res).reset_index().pivot(index='security', columns='scenario').reorder_levels([1, 0], axis=1).sort_index(level=0, axis=1)

In [132]:
bond_df = pd.concat(
    [
        bond_base,
        bond_stress,
    ], axis=1
).dropna()

In [133]:
bond_df = bond_df.set_index(bond_df.index.map(bond_ref.set_index('security')['security_des']))

In [119]:
dg = DataGrid(df)
dg.auto_fit_columns = True
dg

DataGrid(auto_fit_columns=True, auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, corner_render…

#### CDS

In [167]:
cds_mkt_fields = [
    # 'cds_quoted_price',
    'px_last',
    'upfront_last',
]

In [162]:
cds_ref = ref.loc[ref['security_typ'] == 'CREDIT DEFAULT SWAP', :]

In [168]:
cds_pcs = 'MSG1'
cds_sep = ' '

cds_mkt = bq.bdp(
    securities=cds_ref['security'].to_list() if not cds_pcs else add_pcs(cds_ref['security'], source=cds_pcs, sep=cds_sep),
    fields=cds_mkt_fields
)

if cds_pcs:
    cds_mkt['security'] = cds_mkt['security'].str.replace(f'{cds_sep}{cds_pcs}', '')

In [169]:
cds_spread_scenarios = {
    'override_field': 'cds_flat_spread',
    'override_value': 'px_last',
    'scenarios': [
        {
            'name': 'double z',
            'fn': lambda x: x * 2
        },
        {
            'name': 'half z',
            'fn': lambda x: x / 2
        },
    ]
} 

In [170]:
n = 5
cds_res = []
for index, row in cds_mkt.iterrows():
    if index + 1 <= n:
        cds_res.append(get_stressed(row, ['upfront_last'], cds_spread_scenarios))

In [171]:
cds_base = pd.concat([cds_mkt.set_index('security')], keys=['base'], axis=1).rename_axis(['scenario', None], axis=1)
cds_stress = pd.concat(cds_res).reset_index().pivot(index='security', columns='scenario').reorder_levels([1, 0], axis=1).sort_index(level=0, axis=1)

In [172]:
cds_df = pd.concat(
    [
        cds_base,
        cds_stress,
    ], axis=1
).dropna()
cds_df = cds_df.set_index(cds_df.index.map(cds_ref.set_index('security')['security_des']))

In [173]:
cds_df

scenario                     base                     double z               \
                          px_last upfront_last cds_flat_spread upfront_last   
security                                                                      
MEX CDS USD SR 5Y D14      175.50     3.449670          351.00    10.508178   
CHILE CDS USD SR 5Y D14    113.00     0.676974          226.00     5.487739   
TURKEY CDS USD SR 3Y D14   885.00    19.583676         1770.00    34.604442   
BRAZIL CDS USD SR 5Y D14   297.50     8.508604          595.00    19.216255   
EGYPT CDS USD SR 5Y D14   1061.39    32.500000         2122.78    32.500000   

scenario                          half z               
                         cds_flat_spread upfront_last  
security                                               
MEX CDS USD SR 5Y D14             87.750    -0.557753  
CHILE CDS USD SR 5Y D14           56.500    -2.000768  
TURKEY CDS USD SR 3Y D14         442.500     9.041870  
BRAZIL CDS USD SR 5Y D14         148.750     2.176377  
EGYPT CDS USD SR 5Y D14          530.695    32.500000